## Background

Simplified way to interact with some selected Maxar Open Data STAC collections.

Assumes you've run the repo installation instructions and have the docker services running.

## Import libraries

In [ ]:
from datetime import datetime

from eo_maxar.util import MaxarCollection, get_collections

View what collections are available.

In [5]:
get_collections()

['MAXAR_afghanistan_earthquake22',
 'MAXAR_BayofBengal_Cyclone_Mocha_May_23',
 'MAXAR_cyclone_emnati22',
 'MAXAR_Emilia_Romagna_Italy_flooding_may23',
 'MAXAR_Gambia_flooding_8_11_2022',
 'MAXAR_ghana_explosion22',
 'MAXAR_Hurricane_Fiona_9_19_2022',
 'MAXAR_Hurricane_Ian_9_26_2022',
 'MAXAR_Hurricane_Idalia_Florida_Aug23',
 'MAXAR_India_Floods_Oct_2023',
 'MAXAR_Indonesia_Earthquake22',
 'MAXAR_Kahramanmaras_turkey_earthquake_23',
 'MAXAR_Kalehe_DRC_Flooding_5_8_23',
 'MAXAR_kentucky_flooding_7_29_2022',
 'MAXAR_Libya_Floods_Sept_2023',
 'MAXAR_Marshall_Fire_21_Update',
 'MAXAR_Maui_Hawaii_fires_Aug_23',
 'MAXAR_McDougallCreekWildfire_BC_Canada_Aug_23',
 'MAXAR_Morocco_Earthquake_Sept_2023',
 'MAXAR_Nepal_Earthquake_Nov_2023',
 'MAXAR_New_Zealand_Flooding23',
 'MAXAR_NWT_Canada_Aug_23',
 'MAXAR_pakistan_flooding22',
 'MAXAR_shovi_georgia_landslide_8Aug23',
 'MAXAR_southafrica_flooding22',
 'MAXAR_Sudan_flooding_8_22_2022',
 'MAXAR_tonga_volcano21',
 'MAXAR_volcano_indonesia21',
 'MAXA

## Turkey & Syria earthquake

We'll look at the 2023 Turkey & Syria earthquake data to start with.

Define the collection ID, a `MaxarCollection` item to interact with and a pre-post event date.

In [7]:
collection_id = "MAXAR_Kahramanmaras_turkey_earthquake_23"

turkey_earthquake = MaxarCollection(collection_id=collection_id)

event_date = datetime(2023, 2, 6, hour=0, minute=0)  # noqa: DTZ001, RUF100

We can use `get_collection_info()` to return some selected information from the collection.

In [8]:
for attribute, value in vars(turkey_earthquake.get_collection_info()).items():
    print(f"{attribute}: {value}")

id: MAXAR_Kahramanmaras_turkey_earthquake_23
title: Turkey and Syria Earthquake 2023
description: Maxar OpenData | A devastating magnitude 7.8 earthquake struck the Turkish province of Kahramanmaras, approximately 23 kilometers east of Nurdagi in the Gaziantep province near the Syrian border, at 4:17 a.m. local time on Monday, February 6, 2023, followed by a 7.5 magnitude aftershock nine hours later. More than 6,000 people have died in Turkey and Syria, and tens of thousands of people have been injured. Those numbers are expected to increase as search and rescue activities continue. At least 13 million people in the region have been impacted by the earthquake and aftershock. Turkey's president declared a three-month state of emergency in the 10 provinces hardest hit by the earthquake.
extent: {'spatial': {'bbox': [[35.302597, 35.875122, 40.310497, 38.47292570695286], [37.2976, 36.98959965805714, 37.47444448907068, 37.015901889979396], [37.29774464331677, 36.9896650792383, 37.4575240343

### Collection map

We can quickly map the extents of all items within a collection.

The blue boxes specify the extent of each item, whereas the dashed black line is the extent of the entire collection.

In [9]:
m = turkey_earthquake.collection_bbox_map()
m

Map(center=[37.17402385347643, 37.806546999999995], controls=(ZoomControl(options=['position', 'zoom_in_text',…

We can also represent every single item from the collection.

These contain important information elating to the item such as information on the spatio-temporal extent, links to raster data, key metadata and geometry data.

You can find more on the specification: https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md

In [11]:
items = turkey_earthquake.get_collection_items()

### Pre-Post event items
Once we've got the collection items, we can split them into a pre and post event collection.

Blue: pre

Red: post

In [12]:
m = turkey_earthquake.pre_post_map(items, event_date)
m

Map(center=[37.17402385347643, 37.806546999999995], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Single COG image
In our docker services, we have a raster API endpoint connected to the PgSTAC database. The service is built using [titiler-pgstac](http://github.com/stac-utils/titiler-pgstac) and can be used to visualize individual `Item` or `Mosaics` (multiple items).

Raster endpoint: `http://localhost:8082`

Each item has 4 assets and 3 of which are `Cloud-Optimised GeoTIFF` types which is what we're interested in in terms of earth observation data.

Below we'll use the `single_cog_map()` method to view a single item using the `visual` asset - this is the default asset we use.

In [13]:
turkey_earthquake.single_cog_map(items, items[0]["id"])

Map(center=[36.10519545142351, 36.404020998028585], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Mosaic map
Although we can use to `titiler-pgstac` to visualise individual item assets, the raster APIs real benefit is creating a virtual mosaic dynamically and merging multiple items on the fly.

These are created by making `httpx.post` request to the raster API to find all items that match a CQL filter, and are then sorted by the `tile:clouds_percent` and `direction` properties.

We can then use this search request to create the virtual mosaic raster on the fly.

Below we'll create a mosaic for Antakya in the Hatay region before the earthquake.

In [22]:
bounds = [36.129570, 36.180701, 36.194458, 36.224058]

turkey_earthquake.pre_event_mosaic_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 17,
        "layout": {"height": "650px"},
    },
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

And below, we can view it after the earthquake.

In [21]:
turkey_earthquake.post_event_mosaic_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 17,
        "layout": {"height": "650px"},
    },
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

### Mosaic split map
We can also create a split map for a before and after event using the virtual mosaics, making it simpler visualise changes.

In [23]:
turkey_earthquake.mosaic_split_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 17,
        "layout": {"height": "650px"},
    },
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Ghana explosion

We can use the UI frontend to explore other collections, such as the [Ghana explosion](http://localhost:8085/collections/MAXAR_ghana_explosion22).

On 22/01/2022 a motorbike collied with a truck carrying 10 tons of explosives causing a blast which levelled the town of Apiate, located in western Ghana.

We can use the same approach to quickly visalise the impact.


In [16]:
collection_id = "MAXAR_ghana_explosion22"

ghana_explosion = MaxarCollection(collection_id=collection_id)

for attribute, value in vars(ghana_explosion.get_collection_info()).items():
    print(f"{attribute}: {value}")

id: MAXAR_ghana_explosion22
title: Ghana Explosion
description: Maxar OpenData | On Sunday, January 20, 2022, a motorbike collided with a truck carrying 10 tons of explosives causing a blast that leveled the entire town of Apiate, located in western Ghana. The explosives were being delivered to the nearby Chirano gold mine. The small town was reduced to rubble, timber and twisted metal, with a large crater at the center of the explosion. As of January 24, 13 people had been killed and approximately 200 injured.
extent: {'spatial': {'bbox': [[-2.163058656818844, 4.686869783743527, -1.941504778100328, 6.009346352305143], [-2.102893710240898, 4.686869783743527, -1.94443047176199, 6.009346352305143], [-2.163058656818844, 5.24370462781881, -1.993741099332081, 5.664113390366951], [-2.11652117657606, 5.475347139049133, -1.941504778100328, 5.670042897561462]]}, 'temporal': {'interval': [['2020-01-06T10:43:48Z', '2022-01-22T10:49:54Z']]}}
links: [{'rel': 'items', 'type': 'application/geo+json',

We can use tools from `geopandas` to geocode areas and make it a little easier for constructing the map requirements.

In [ ]:
import geopandas as gpd

gdf = gpd.tools.geocode("Apiate, Ghana")

bounds = list(
    map(
        float, gdf.to_crs(gdf.estimate_utm_crs()).buffer(1000).to_crs(4326).total_bounds
    )
)

event_date = datetime(2022, 1, 19, hour=0, minute=0)  # noqa: DTZ001, RUF100

items = ghana_explosion.get_collection_items()

In [18]:
ghana_explosion.mosaic_split_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 18,
        "layout": {"height": "700px"},
    },
)

Map(center=[5.584911298591983, -2.0464450120290216], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Hurricane Ian

Another example from the [Hurricane Ian collection](http://localhost:8085/collections/MAXAR_Hurricane_Ian_9_26_2022).

Hurricane Ian made landfall as a Category 4 storm near Cay Costa, Florida, on Wednesday, September 28, 2022, with winds of 150 mph and with record storm surge flooding as high as 12 feet in some coastal areas.

We'll have a quick look at the before and after on Sanibel Island, Florida.

In [24]:
collection_id = "MAXAR_Hurricane_Ian_9_26_2022"

hurricane_ian = MaxarCollection(collection_id=collection_id)

for attribute, value in vars(hurricane_ian.get_collection_info()).items():
    print(f"{attribute}: {value}")

id: MAXAR_Hurricane_Ian_9_26_2022
title: Hurricane Ian
description: Maxar OpenData | Hurricane Ian made landfall as a Category 4 storm near Cay Costa, Florida, on Wednesday, September 28, 2022, with winds of 150 mph and with record storm surge flooding as high as 12 feet in some coastal areas. 2.5 million people were left without power while property damages were estimated to be at least $113 billion. 116 people were killed in the USA and neighboring Cuba.
extent: {'spatial': {'bbox': [[-83.901156, 22.179453, -78.76563736022848, 33.96944669284327], [-82.0198065782888, 26.464945002172342, -81.74089679642375, 26.88739117754846], [-81.98115622654164, 25.999567441531866, -81.6012435447469, 26.601251377033204], [-81.69349032663095, 25.998435318619297, -81.47416523669433, 26.630565603111673], [-81.91307486127658, 26.310667782819692, -81.73505446273649, 26.47069233481796], [-82.159635401552, 26.063780134370397, -81.97508024877337, 27.013933488312134], [-82.26627444807235, 26.389276848311134, 

In [ ]:
gdf = gpd.tools.geocode("Sanibel Island, Florida")

bounds = list(
    map(
        float, gdf.to_crs(gdf.estimate_utm_crs()).buffer(5000).to_crs(4326).total_bounds
    )
)

event_date = datetime(2022, 9, 28, hour=0, minute=0)  # noqa: DTZ001, RUF100

items = hurricane_ian.get_collection_items()

In [26]:
items = hurricane_ian.get_collection_items()

Post-event mosaic.

In [23]:
hurricane_ian.post_event_mosaic_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 12,
        "layout": {"height": "650px"},
    },
)

Map(center=[26.440358854138267, -82.1137043870914], controls=(ZoomControl(options=['position', 'zoom_in_text',…